In [1]:
import torch as th
from sb3_contrib import TQC
from active_critic.utils.gym_utils import make_policy_dict, make_dummy_vec_env
from stable_baselines3.common.torch_layers import CombinedExtractor
from stable_baselines3.common.evaluation import evaluate_policy
import gym
import numpy as np
from gym import Env

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [2]:
seq_env, expert = make_dummy_vec_env(name='reach', seq_len=100)
policy_kwargs = dict(n_critics=2, n_quantiles=25, net_arch=[512,512,512])

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [3]:
tqc_learner = TQC(policy='MlpPolicy',env=seq_env, top_quantiles_to_drop_per_net=2, verbose=1, policy_kwargs=policy_kwargs)

Using cuda device


In [4]:
tqc_learner.learn(50000, log_interval=float('inf'))

In [5]:
seq_env.envs[0].reset_count

501

In [9]:
obsv = seq_env.reset()
rews = []
success = []
for i in range(10000):
    action = tqc_learner.policy.predict(obsv[0])
    obsv, rew, done, info = seq_env.step(action)
    rews.append(rew)
    if done[0]:
        success.append(info[0]['success'])

In [10]:
rews = np.array(rews)
success = np.array(success)
print(f'rew: {rews.mean()}')
print(f'success: {success.mean()}')


rew: 8.023058891296387
success: 0.31


In [8]:
success

array([0., 0., 0., 1., 1., 1., 0., 0., 1., 0.])